In [ ]:
!pip install autokeras


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import autokeras as ak


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
directory = '/content/drive/MyDrive/sea_turtle_images'

In [ ]:
import pathlib
data_categ = tf.keras.utils.get_file(origin=directory, 
                                   fname='/content/drive/MyDrive/sea_turtle_images', 
                                   untar=True)
data_categ = pathlib.Path(data_dir)

In [ ]:
image_count = len(list(data_categ.glob('*')))
print(image_count)

4


## A Simple Example
The first step is to prepare your data. Here we use the MNIST dataset as an example


In [ ]:
# (x_train, y_train), (x_test, y_test) = train_ds, val_ds
# print(x_train.shape)  # (60000, 28, 28)
# print(y_train.shape)  # (60000,)
# print(y_train[:3])  # array([7, 2, 1], dtype=uint8)

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

train_data = ak.image_dataset_from_directory(
    directory,
    # Use 20% data as testing data.
    validation_split=0.2,
    subset="training",
    # Set seed to ensure the same split when loading testing data.
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)

test_data = ak.image_dataset_from_directory(
    directory,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)

Found 55 files belonging to 4 classes.
Using 44 files for training.
Found 55 files belonging to 4 classes.
Using 11 files for validation.


The second step is to run the ImageClassifier.
It is recommended have more trials for more complicated datasets.
This is just a quick demo of MNIST, so we set max_trials to 1.
For the same reason, we set epochs to 10.
You can also leave the epochs unspecified for an adaptive number of epochs.


In [ ]:
# Initialize the image classifier.
clf = ak.ImageClassifier(overwrite=True, max_trials=1)
# Feed the image classifier with training data.
clf.fit(train_data, epochs=10)


# Predict with the best model.
predicted_y = clf.predict(test_data)
print(predicted_y)


# Evaluate the best model with testing data.
print(clf.evaluate(test_data))


Trial 1 Complete [00h 00m 49s]
val_loss: 5.372154235839844

Best val_loss So Far: 5.372154235839844
Total elapsed time: 00h 00m 49s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
2/2 [==============================] - 6s 1s/step - loss: 8.7398 - accuracy: 0.2500
Epoch 2/10
2/2 [==============================] - 5s 1s/step - loss: 9.9975 - accuracy: 0.3409
Epoch 3/10
2/2 [==============================] - 5s 1s/step - loss: 3.1376 - accuracy: 0.6591
Epoch 4/10
2/2 [==============================] - 5s 1s/step - loss: 4.2618 - accuracy: 0.5682
Epoch 5/10
2/2 [==============================] - 5s 1s/step - loss: 2.2467 - accuracy: 0.7727
Epoch 6/10
2/2 [==============================] - 5s 1s/step - loss: 0.9104 - accuracy: 0.8636
Epoch 7/10
2/2 [==============================] - 5s 1s/step - loss: 0.1269 - accuracy: 0.9545
Epoch 8/10
2/2 [==============================] - 5s 1s/step - loss: 0.2231 - accuracy: 0.8864
Epoch 9/10
2/2 [==============================] - 5s 1s/step - loss: 0

## Validation Data
By default, AutoKeras use the last 20% of training data as validation data. As
shown in the example below, you can use validation_split to specify the
percentage.


In [ ]:
clf.fit(
    train_data,
    # Split the training data and use the last 15% as validation data.
    validation_split = 0.15,
    epochs=10,
)


You can also use your own validation set instead of splitting it from the
training data with validation_data.


In [ ]:
# split = 50000
# x_val = x_train[split:]
# y_val = y_train[split:]
# x_train = x_train[:split]
# y_train = y_train[:split]
# clf.fit(
#     x_train,
#     y_train,
#     # Use your own validation set.
#     validation_data=(x_val, y_val),
#     epochs=10,
# )


## Customized Search Space
For advanced users, you may customize your search space by using AutoModel
instead of ImageClassifier. You can configure the ImageBlock for some
high-level configurations, e.g., block_type for the type of neural network to
search, normalize for whether to do data normalization, augment for whether to
do data augmentation. You can also do not specify these arguments, which would
leave the different choices to be tuned automatically. See the following
example for detail.


In [ ]:
input_node = ak.ImageInput()
output_node = ak.ImageBlock(
    # Only search ResNet architectures.
    block_type="resnet",
    # Normalize the dataset.
    normalize=True,
    # Do not do data augmentation.
    augment=False,
)(input_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)
clf.fit(train_data, epochs=10)


Trial 1 Complete [00h 06m 08s]
val_loss: 8.098440170288086

Best val_loss So Far: 8.098440170288086
Total elapsed time: 00h 06m 08s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
2/2 [==============================] - 43s 9s/step - loss: 5.8251 - accuracy: 0.3182
Epoch 2/10
2/2 [==============================] - 32s 9s/step - loss: 11.5015 - accuracy: 0.4318
Epoch 3/10
2/2 [==============================] - 32s 9s/step - loss: 2.6947 - accuracy: 0.5909
Epoch 4/10
2/2 [==============================] - 32s 9s/step - loss: 4.5318 - accuracy: 0.3182
Epoch 5/10
2/2 [==============================] - 32s 9s/step - loss: 0.7822 - accuracy: 0.7955
Epoch 6/10
2/2 [==============================] - 32s 9s/step - loss: 2.4464 - accuracy: 0.6364
Epoch 7/10
2/2 [==============================] - 32s 9s/step - loss: 0.9353 - accuracy: 0.7727
Epoch 8/10
2/2 [==============================] - 32s 9s/step - loss: 0.6856 - accuracy: 0.7955
Epoch 9/10
2/2 [==============================] - 32s 9s/step

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


The usage of AutoModel is similar to the functional API of Keras. Basically, you are
building a graph, whose edges are blocks and the nodes are intermediate outputs of
blocks. To add an edge from input_node to output_node with output_node =
ak.[some_block]([block_args])(input_node).

You can even also use more fine grained blocks to customize the search space even
further. See the following example.


In [ ]:
input_node = ak.ImageInput()
output_node = ak.Normalization()(input_node)
output_node = ak.ImageAugmentation(horizontal_flip=False)(output_node)
output_node = ak.ResNetBlock(version="v2")(output_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)
clf.fit(train_data, epochs=10)


Trial 1 Complete [00h 03m 03s]
val_loss: 3.034581184387207

Best val_loss So Far: 3.034581184387207
Total elapsed time: 00h 03m 03s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
2/2 [==============================] - 23s 5s/step - loss: 9.5519 - accuracy: 0.2955
Epoch 2/10
2/2 [==============================] - 18s 5s/step - loss: 18.4309 - accuracy: 0.3636
Epoch 3/10
2/2 [==============================] - 18s 5s/step - loss: 6.4764 - accuracy: 0.4545
Epoch 4/10
2/2 [==============================] - 18s 5s/step - loss: 3.3055 - accuracy: 0.5227
Epoch 5/10
2/2 [==============================] - 18s 5s/step - loss: 2.6301 - accuracy: 0.4545
Epoch 6/10
2/2 [==============================] - 18s 5s/step - loss: 1.1067 - accuracy: 0.4773
Epoch 7/10
2/2 [==============================] - 17s 5s/step - loss: 0.8480 - accuracy: 0.7273
Epoch 8/10
2/2 [==============================] - 18s 5s/step - loss: 1.1000 - accuracy: 0.7500
Epoch 9/10
2/2 [==============================] - 17s 5s/step

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


## Data Format
The AutoKeras ImageClassifier is quite flexible for the data format.

For the image, it accepts data formats both with and without the channel
dimension. The images in the MNIST dataset do not have the channel dimension.
Each image is a matrix with shape (28, 28). AutoKeras also accepts images of
three dimensions with the channel dimension at last, e.g., (32, 32, 3), (28,
28, 1).

For the classification labels, AutoKeras accepts both plain labels, i.e.
strings or integers, and one-hot encoded encoded labels, i.e. vectors of 0s and
1s.

So if you prepare your data in the following way, the ImageClassifier should
still work.


In [ ]:
# (x_train, y_train), (x_test, y_test) = train_data, test_data

# # Reshape the images to have the channel dimension.
# x_train = x_train.reshape(x_train.shape + (1,))
# x_test = x_test.reshape(x_test.shape + (1,))

# # One-hot encode the labels.
# eye = np.eye(10)
# y_train = eye[y_train]
# y_test = eye[y_test]

# print(x_train.shape)  # (60000, 28, 28, 1)
# print(y_train.shape)  # (60000, 10)
# print(y_train[:3])
# # array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
# #        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
# #        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])


We also support using tf.data.Dataset format for the training data.


In [ ]:
# train_set = tf.data.Dataset.from_tensor_slices(((x_train,), (y_train,)))
# test_set = tf.data.Dataset.from_tensor_slices(((x_test,), (y_test,)))

# clf = ak.ImageClassifier(overwrite=True, max_trials=1)
# # Feed the tensorflow Dataset to the classifier.
# clf.fit(train_set, epochs=10)
# # Predict with the best model.
# predicted_y = clf.predict(test_set)
# # Evaluate the best model with testing data.
# print(clf.evaluate(test_set))


## Reference
[ImageClassifier](/image_classifier),
[AutoModel](/auto_model/#automodel-class),
[ImageBlock](/block/#imageblock-class),
[Normalization](/block/#normalization-class),
[ImageAugmentation](/block/#image-augmentation-class),
[ResNetBlock](/block/#resnetblock-class),
[ImageInput](/node/#imageinput-class),
[ClassificationHead](/block/#classificationhead-class).
